In [ ]:
import json, numbers, math, time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from elfpy.simulators import YieldSimulator
from elfpy.utils.data import format_trades

In [ ]:
pd.set_option("float_format",'{:,.6f}'.format)

random_seed = 3
simulator_rng = np.random.default_rng(random_seed)
config = {
    "min_fee": 0.1,
    "max_fee": 0.5,
    "min_target_liquidity": 1e6,
    "max_target_liquidity": 10e6,
    "min_target_volume": 0.01, # fraction of pool liquidity
    "max_target_volume": 0.2, # fration of pool liquidity
    "min_pool_apy": 0.01,  # as a decimal
    "max_pool_apy": 0.9,  # as a decimal
    "min_vault_age": 0,  # fraction of a year
    "max_vault_age": 1,  # fraction of a year
    "min_vault_apy": 0.001,  # as a decimal
    "max_vault_apy": 0.9,  # as a decimal
    "base_asset_price": 2500.0,  # aka market price
    "pool_duration": 180,
    "num_trading_days": 180,  # should be <= pool_duration
    "floor_fee": 0, # minimum fee percentage
    "tokens": ["base", "fyt"],
    "trade_direction": "out",
    "precision": None,
    "pricing_model_name": "Element",
    "rng": simulator_rng,
    "verbose": False,
}

In [ ]:
start_time = time.time()
num_runs = 0

simulator = YieldSimulator(**config)
simulator.set_random_variables()

apy = [0.10,]*config['num_trading_days']
#apy[config['num_trading_days']//2:(config['num_trading_days']//2) + 5] == [0.08, 0.08, 0.08]
#apy[config['num_trading_days']//2:] = [0.08,]*(config['num_trading_days']//2)
#apy[config['num_trading_days']//2:] = [0.01,]*(config['num_trading_days']//2)

override_dict = {
    'pricing_model_name': 'Element',
    'vault_apy': apy,
    'init_pool_apy' : apy[0],
    'init_vault_age': 1,
    'target_daily_volume': 5*1e5,
    'target_liquidity': 10*1e6,
    'fee_percent' : 0.1,
}
simulator.reset_rng(np.random.default_rng(random_seed)) # do this to make sure simulations run over the same trade sequence
simulator.run_simulation(override_dict)
num_runs += 1

floor_fee_list = [0, 5]
override_dict['pricing_model_name'] = 'YieldSpacev2'
for floor_fee in floor_fee_list:
    override_dict['floor_fee'] = floor_fee
    simulator.reset_rng(np.random.default_rng(random_seed)) # do this to make sure simulations run over the same trade sequence
    simulator.run_simulation(override_dict)
    num_runs += 1

end_time = time.time()
print(f'Total time for {num_runs} runs was {end_time-start_time:.3f} seconds; which is {(end_time-start_time)/num_runs:.3f} seconds per run')

In [ ]:
[trades, trades_agg] = format_trades(simulator.analysis_dict)

In [ ]:
display(trades.groupby(['model_name', 'token_in']).agg({
    'trade_volume_usd': ['sum'],
    'run_trade_number': ['count'],
    'fee_in_bps': ['mean', 'std', 'min', 'max', 'sum'],
    'day': ['mean', 'min', 'max'],
    })
)

In [ ]:
numPlots = 2
colors = list(plt.rcParams['axes.prop_cycle'].by_key()['color'])
model_colors = {model:colors[i] for i, model in enumerate(trades_agg.model_name.unique())}

## DAY PLOTS
figsize = (24, 12)
fig, ax = plt.subplots(
    ncols=1,
    nrows=numPlots,
    gridspec_kw={'wspace':0, 'hspace':0, 'height_ratios':np.ones(numPlots)}
)
fig.patch.set_facecolor('white')   # set fig background color to white

# fees
current_plot = 0
for model in trades_agg.model_name.unique():
    model_df = trades_agg.loc[trades_agg.model_name==model, :]
    ax[current_plot] = model_df.plot(
        x='day',
        y='fee_in_usd_sum',
        figsize=figsize,
        ax=ax[current_plot],
        color=model_colors[model],
        label=model
    )
ax[current_plot].set_xlabel('')
ax[current_plot].set_ylabel('Fees (USD)', fontsize=18)
ax[current_plot].tick_params(axis='both', labelsize=18)
ax[current_plot].grid(
    visible=True, linestyle='--', linewidth='1', color='grey', which='both', axis='y'
)
ax[current_plot].xaxis.set_ticklabels([])
title = (
    f'Initial pool APY: {trades.pool_apy[0]:.2f}% '
    + f'Time Stretch: {trades.init_time_stretch[0]:.2f} '
    + f'Maturity: {trades.pool_duration[0]} days\n'
    + f'Target Liquidity: {trades.target_liquidity[0]:,.0f} '
    + f'Target Daily Volume: {trades.target_daily_volume[0]:,.0f} '
    + f'Percent Fees: {trades.fee_percent[0]:.2f}%'
)
ax[current_plot].set_title(title, fontsize=20)
ax[current_plot].legend(fontsize=18)

# fees cumulative
current_plot += 1
for model_idx, model in enumerate(trades_agg.model_name.unique()):
    model_df = trades_agg.loc[trades_agg.model_name==model, :]
    #cumulative_fee = model_df.fee_in_usd_cum_sum.iloc[-1]
    ax[current_plot] = model_df.plot(
        x='day',
        y='fee_in_usd_cum_sum',
        figsize=figsize,
        ax=ax[current_plot],
        color=model_colors[model],
        legend=None,
    )
    total_fees = model_df.fee_in_usd_cum_sum.iloc[-1]
    ax[current_plot].annotate(
        f'{model} total fees collected = {total_fees:.2f} USD',
        (125, 1.006e5-2.45e4*model_idx),
        fontsize=14
    )
ax[current_plot].set_ylabel('Cumulative Fees (USD)', fontsize=18)
ax[current_plot].tick_params(axis='both', labelsize=18)
ax[current_plot].grid(
    visible=True, linestyle='--', linewidth='1', color='grey', which='both', axis='y'
)
ax[current_plot].set_xlabel('Day', fontsize=18)

if apy[0] == apy[-1]:
    fig.savefig('../outputs/fees_constant_vault_apy.png')
else:
    fig.savefig('../outputs/fees_variable_vault_apy.png')

In [ ]:
## TRADE PLOTS
num_plots = 2
figsize = (24, 12)
fig, ax = plt.subplots(
    ncols=1,
    nrows=numPlots,
    gridspec_kw={'wspace':0, 'hspace':0, 'height_ratios':np.ones(numPlots)}
)
fig.patch.set_facecolor('white')   # set fig background color to white

# pool & vault APYs
current_plot = 0
for model_idx, model in enumerate(trades.model_name.unique()): # for each model (per run?)
    model_df = trades.loc[trades.model_name==model, :]
    ax[current_plot] = model_df.plot(
        x='run_trade_number', # could also do num_orders if you want to know the apy for a given volume of trade
        y='pool_apy_percent',
        figsize=figsize,
        ax=ax[current_plot],
        color=model_colors[model],
        label=f'{model}'
    )
    if model_idx == len(trades.model_name.unique()) - 1:
        label = 'vault'
    else:
        label = '_nolegend_'
    ax[current_plot] = model_df.plot(
        x='run_trade_number',
        y='vault_apy_percent',
        figsize=figsize,
        ax=ax[current_plot],
        linestyle='-',
        color='black',
        label=label,
    )
ax[current_plot].set_xlabel('')
ax[current_plot].set_ylabel('APY (percent)', fontsize=18)
ax[current_plot].tick_params(axis='both', labelsize=18)
ax[current_plot].grid(
    visible=True, linestyle='--', linewidth='1', color='grey', which='both', axis='y'
)
ax[current_plot].xaxis.set_ticklabels([])
ax[current_plot].legend(fontsize=18)


# price per share & spot price
current_plot += 1
for model_idx, model in enumerate(trades.model_name.unique()):
    model_df = trades.loc[trades.model_name==model, :]
    share_price_growth = (
        model_df.loc[:, "share_price"].values[-1]
        / model_df.loc[:, "share_price"].values[0]
        - 1
    )
    ax[current_plot] = model_df.plot(
        x='run_trade_number',
        y='share_price',
        figsize=figsize,
        ax=ax[current_plot],
        kind='line',
        style='--',
        color=model_colors[model],
        label='Share Price'
    )
    ax[current_plot].annotate(
        f'{model} share price growth = {share_price_growth:.3%}',
        (1400, 1.126-0.0105*model_idx),
        fontsize=14
    )

    spot_price_growth = (
        model_df.loc[:, "pu"].values[-1]
        / model_df.loc[:, "pu"].values[0]
        - 1
    )
    ax[current_plot] = model_df.plot(
        x='run_trade_number',
        y='pu',
        figsize=figsize,
        ax=ax[current_plot],
        kind='line',
        color=model_colors[model],
        label='Spot Price'
    )
    ax[current_plot].annotate(
        f'{model} spot price growth = {spot_price_growth:.3%}',
        (1400, 1.122-0.0105*model_idx),
        fontsize=14
    )

ax[current_plot].set_ylabel('Pricing', fontsize=18)
ax[current_plot].tick_params(axis='both', labelsize=18)
ax[current_plot].grid(visible=True, linestyle='--', linewidth='1', color='grey', which='both', axis='y')
h,l = ax[current_plot].get_legend_handles_labels()
ax[current_plot].legend([l[0], l[1]], fontsize=18)
ax[current_plot].set_xlabel('Trade number', fontsize=18)

if apy[0] == apy[-1]:
    fig.savefig('../outputs/pricing_constant_vault_apy.png')
else:
    fig.savefig('../outputs/pricing_variable_vault_apy.png')

plt.show()